In [1]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import autosklearn.classification
import os
import featuretools as ft
import warnings
import autofeat 
from tpot import TPOTClassifier
import json 
import time
from sklearn import preprocessing 


warnings.filterwarnings('ignore')

/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [41]:
def run_as(X, y, target_ft, time_budget=30, include_preprocessors = None, n_jobs=-1):
    try:
        os.remove('/tmp/autosklearn_regression_example_tmp')
        os.remove('/tmp/autosklearn_regression_example_out')
    except:
        pass
    #X = df.drop(columns=target_ft)
    #y = df[target_ft]
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)
    automl = autosklearn.classification.AutoSklearnClassifier(
        time_left_for_this_task=time_budget,
        #per_run_time_limit=30,
        tmp_folder='./tmp/autosklearn_regression_example_tmp',
        output_folder='./tmp/autosklearn_regression_example_out',
        include_preprocessors=include_preprocessors,
        ml_memory_limit=None,
        ensemble_memory_limit=None,
        metric=autosklearn.metrics.f1_weighted,
        n_jobs=n_jobs
    )
    automl.fit(X_train, y_train)
    y_hat = automl.predict(X_test)
    
    acc = sklearn.metrics.accuracy_score(y_test, y_hat)
    f1_s = sklearn.metrics.f1_score(y_test, y_hat, average='weighted')
    metrs = []
    metrs.append("Accuracy score - " + str(acc))
    metrs.append("F1 score - " + str(f1_s))
    
    res = ["","","","",f1_s,acc,"",automl.show_models()]
    
    
    return str(metrs),res

def run_tpot(X,y, target_ft,time_budget=30, include_preprocessors=None ):
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)
    pipeline_optimizer = TPOTClassifier(max_time_mins = time_budget/60, generations=None)
    pipeline_optimizer.fit(X_train, y_train)
    y_hat = pipeline_optimizer.predict(X_test)
    metrs = []
    metrs.append("Accuracy score - " + str(sklearn.metrics.accuracy_score(y_test, y_hat)))
    metrs.append("F1 score - " + str(sklearn.metrics.f1_score(y_test, y_hat, average='macro')))
    return str(metrs)

    
def gen_feats_featools(df):
    es = ft.EntitySet(id = 'df')
    es.entity_from_dataframe(entity_id = 'data', dataframe = df, 
                         make_index = True, index = 'index')
    feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'data', verbose = 1)
                                      #agg_primitives=["mean", "max", "min", "std", "skew"],
                                     # trans_primitives = ['add_numeric', 'multiply_numeric'])
    return feature_matrix

def gen_feats_autofeat(X,y):
    fsel = autofeat.FeatureSelector(verbose=1)
    X = fsel.fit_transform(X,y)
    return X
    
    
def run_test(df_path,target_ft, mode = 0, time_budget=30,n_jobs=-1):
    results = []
    df = pd.read_csv(df_path)
    object_columns = df.select_dtypes(include='object')
    if len(object_columns.columns):
        df[object_columns.columns] = object_columns.apply(preprocessing.LabelEncoder().fit_transform)
    X = df.drop(columns=target_ft)
    y = df[target_ft]
    res_df = []
    autofeat_time = 0
    
    if mode == 0 or mode == 1:
        start = time.monotonic()
        X_new = gen_feats_autofeat(X,y)
        end = time.monotonic()
        autofeat_time = int(end-start)
        print("Autofeat_time: ",autofeat_time)
        rs = run_as(X_new,y,target_ft,time_budget=time_budget, include_preprocessors =["no_preprocessing"],n_jobs=n_jobs)
        results.append("Autosk Only with Preprocessing: " + rs[0])
        rs[1][0] = df_path[5:-4]
        rs[1][1] = str(time_budget/60)+'m'
        rs[1][2] = "Autofeat"
        rs[1][3] = "AutoSK"
        rs[1][6] = str(X_new.shape)
        res_df.append(rs[1])
    

    if mode == 0 or mode == 2:
        rs = run_as(X,y,target_ft, time_budget=time_budget+autofeat_time, include_preprocessors=None,n_jobs=n_jobs)   
        results.append("Autosk Only with Preprocessing: " + rs[0])
        rs[1][0] = df_path[5:-4]
        rs[1][1] = str(round((time_budget+autofeat_time)/60,2))+'m'
        rs[1][2] = "AutoSK"
        rs[1][3] = "AutoSK"
        rs[1][6] = str(X.shape)
        res_df.append(rs[1])
    if mode == 0 or mode == 3:
        rs = run_as(X,y,target_ft,time_budget=time_budget, include_preprocessors =["no_preprocessing"],n_jobs=n_jobs)
        results.append("Autosk Only without Preprocessing: " + rs[0])
        rs[1][0] = df_path[5:-4]
        rs[1][1] = str(round((time_budget+autofeat_time)/60,2))+'m'
        rs[1][2] = "None"
        rs[1][3] = "AutoSK"
        rs[1][6] = str(X.shape)
        res_df.append(rs[1])
    
    
    
    
    #if mode == 0 or mode == 4:
    #    X_new = gen_feats_featools(X)
    #    rs = run_as(X_new,y,target_ft,time_budget=time_budget, include_preprocessors =["no_preprocessing"])
    #    results.append("Autosk with Featuretools: " + rs)

    #if mode ==0 or mode == 5:
    #    rs = run_tpot(X,y,target_ft, time_budget=time_budget, include_preprocessors=None)   
    #    results.append("TPOT Only with Preprocessing: " + rs)
    
    
        
    print("===================================")
    print("Time budeget: ",time_budget)
    [print(x) for x in results]
    
    res_df =  pd.DataFrame(res_df, columns = ["Dataset","Time","Preprocessing","AutoML","Accuracy","F1","Shape","PipeLine"])
    res_df.drop(columns=["PipeLine"]).to_csv("results/"+df_path[5:])
    res_df.to_csv("results/pipe_"+df_path[5:])
    
    return res_df

In [47]:
!rm -r tmp
df_path = "data/gina.csv"
target_ft = "class"
res = run_test(df_path, target_ft, mode=2 ,n_jobs=-1,time_budget=120)

Exception ignored in: <bound method BackendContext.__del__ of <autosklearn.util.backend.BackendContext object at 0x7facc5ce08d0>>
Traceback (most recent call last):
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/util/backend.py", line 128, in __del__
    self.delete_directories(force=False)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/util/backend.py", line 136, in delete_directories
    % self.output_directory)
ValueError: Failed to delete output dir: ./tmp/autosklearn_regression_example_out because auto-sklearn did not create it. Please make sure that the specified output dir does not exist when instantiating auto-sklearn.
Exception ignored in: <bound method BackendContext.__del__ of <autosklearn.util.backend.BackendContext object at 0x7facc5ce08d0>>
Traceback (most recent call last):
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/util/backend.py", line 128, in __del__
    self.delete_di

[WARNING] [2020-08-04 12:52:39,504:AutoML(3645644232):5193c1c86d0dd70aa3b0a1499d7fd4e0] Capping the per_run_time_limit to 29.0 to have time for a least 2 models in each process.


Exception ignored in: <bound method BackendContext.__del__ of <autosklearn.util.backend.BackendContext object at 0x7facc5ce08d0>>
Traceback (most recent call last):
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/util/backend.py", line 128, in __del__
    self.delete_directories(force=False)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/util/backend.py", line 136, in delete_directories
    % self.output_directory)
ValueError: Failed to delete output dir: ./tmp/autosklearn_regression_example_out because auto-sklearn did not create it. Please make sure that the specified output dir does not exist when instantiating auto-sklearn.
Exception ignored in: <bound method BackendContext.__del__ of <autosklearn.util.backend.BackendContext object at 0x7facc5ce08d0>>
Traceback (most recent call last):
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/util/backend.py", line 128, in __del__
    self.delete_di

[WARNING] [2020-08-04 12:52:46,759:AutoML(3636387353):5193c1c86d0dd70aa3b0a1499d7fd4e0] Capping the per_run_time_limit to 27.0 to have time for a least 2 models in each process.


Exception ignored in: <bound method BackendContext.__del__ of <autosklearn.util.backend.BackendContext object at 0x7facc5ce08d0>>
Traceback (most recent call last):
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/util/backend.py", line 128, in __del__
    self.delete_directories(force=False)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/util/backend.py", line 136, in delete_directories
    % self.output_directory)
ValueError: Failed to delete output dir: ./tmp/autosklearn_regression_example_out because auto-sklearn did not create it. Please make sure that the specified output dir does not exist when instantiating auto-sklearn.
Exception ignored in: <bound method BackendContext.__del__ of <autosklearn.util.backend.BackendContext object at 0x7facc5ce08d0>>
Traceback (most recent call last):
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/util/backend.py", line 128, in __del__
    self.delete_di

...

[WARNING] [2020-08-04 12:53:37,602:smac.runhistory.runhistory.RunHistory] Encountered exception Expecting value: line 1 column 1 (char 0) while reading runhistory from ./tmp/autosklearn_regression_example_tmp/smac3-output/run_2793336106/runhistory.json. Not adding any runs!
[WARNING] [2020-08-04 12:53:39,440:smac.runhistory.runhistory.RunHistory] Encountered exception Expecting value: line 1 column 1 (char 0) while reading runhistory from ./tmp/autosklearn_regression_example_tmp/smac3-output/run_1067342868/runhistory.json. Not adding any runs!


Exception ignored in: <bound method BackendContext.__del__ of <autosklearn.util.backend.BackendContext object at 0x7facc5ce08d0>>
Traceback (most recent call last):
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/util/backend.py", line 128, in __del__
    self.delete_directories(force=False)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/util/backend.py", line 136, in delete_directories
    % self.output_directory)
ValueError: Failed to delete output dir: ./tmp/autosklearn_regression_example_out because auto-sklearn did not create it. Please make sure that the specified output dir does not exist when instantiating auto-sklearn.
Exception ignored in: <bound method BackendContext.__del__ of <autosklearn.util.backend.BackendContext object at 0x7facc5ce08d0>>
Traceback (most recent call last):
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/util/backend.py", line 128, in __del__
    self.delete_di

[WARNING] [2020-08-04 12:53:42,351:smac.runhistory.runhistory.RunHistory] Encountered exception Expecting value: line 1 column 1 (char 0) while reading runhistory from ./tmp/autosklearn_regression_example_tmp/smac3-output/run_2124297904/runhistory.json. Not adding any runs!
[ERROR] [2020-08-04 12:53:46,301:AutoML(4227071829):5193c1c86d0dd70aa3b0a1499d7fd4e0] %d format: a number is required, not NoneType
Traceback (most recent call last):
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/evaluation/__init__.py", line 265, in run
    info = autosklearn.evaluation.util.read_queue(queue)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/evaluation/util.py", line 30, in read_queue
    raise queue.Empty
queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 485, in fit
    _proc_

Process Process-580:
Traceback (most recent call last):
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/evaluation/__init__.py", line 265, in run
    info = autosklearn.evaluation.util.read_queue(queue)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/evaluation/util.py", line 30, in read_queue
    raise queue.Empty
queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/estimators.py", line 16, in _fit_automl
    return automl.fit(load_models=load_models, **kwargs)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 1037, in fit
    lo

Time budeget:  120
Autosk Only with Preprocessing: ['Accuracy score - 0.9252217997465145', 'F1 score - 0.9252261331445735']


In [38]:
#!rm -r tmp
#df = pd.read_csv("blood.csv")
#target_ft = "class"
#run_test(df, target_ft, mode=2, time_budget=60)

In [7]:
#df = pd.read_csv("winequality-red.csv")
#target_ft = "quality"
#run_test(df, target_ft, mode=2 ,time_budget=30)

In [6]:
#df = pd.read_csv("data/airlines.csv")

In [7]:
#!rm -r tmp
#df = pd.read_csv("data/airlines.csv").drop(columns=["Airline","AirportFrom","AirportTo"])
#target_ft = "Delay"
#run_test(df, target_ft, mode=2 ,time_budget=30)

In [50]:
#!rm -r tmp
#df = "data/gina.csv"
#target_ft = "class"
#run_test(df, target_ft, mode=2 ,time_budget=30)

In [9]:
#!rm -r tmp
#df = pd.read_csv("data/gina.csv")
#target_ft = "class"
#run_test(df, target_ft, mode=4 ,time_budget=420)

In [16]:
#!ls data

20_newsgroups.csv   ailerons.csv	     eeg-eye-state.csv	nomao.csv
Christine.csv	    banknote.csv	     enron.csv
KDDCup09_churn.csv  dataset_44_spambase.csv  gina.csv


In [49]:
#!rm -r tmp
df_path = "data/20_newsgroups.csv"
target_ft = "class"
res = run_test(df_path, target_ft, mode=2 ,time_budget=120, n_jobs=-1)

[WARNING] [2020-08-04 13:03:30,480:AutoML(4123242914):f206b5794857f169f2b9da4c9d08922d] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (12.422594)
[WARNING] [2020-08-04 13:03:30,485:AutoML(4123242914):f206b5794857f169f2b9da4c9d08922d] Capping the per_run_time_limit to 6.0 to have time for a least 2 models in each process.
[ERROR] [2020-08-04 13:03:52,436:AutoML(1872583848):f206b5794857f169f2b9da4c9d08922d] Error creating dummy predictions: {'error': 'Timeout', 'configuration_origin': 'DUMMY'} 


Process Process-658:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/estimators.py", line 16, in _fit_automl
    return automl.fit(load_models=load_models, **kwargs)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 1037, in fit
    load_models=load_models,
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 365, in fit
    num_run = self._do_dummy_prediction(datamanager, num_run)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 265, in _do_dummy_prediction
    % (str(status), str(additional_info)))
ValueError: Dummy prediction failed with run state StatusType.TIMEOUT and addit

[ERROR] [2020-08-04 13:04:18,819:AutoML(3636387353):f206b5794857f169f2b9da4c9d08922d] Error creating dummy predictions: {'error': 'Timeout', 'configuration_origin': 'DUMMY'} 


Process Process-709:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/estimators.py", line 16, in _fit_automl
    return automl.fit(load_models=load_models, **kwargs)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 1037, in fit
    load_models=load_models,
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 365, in fit
    num_run = self._do_dummy_prediction(datamanager, num_run)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 265, in _do_dummy_prediction
    % (str(status), str(additional_info)))
ValueError: Dummy prediction failed with run state StatusType.TIMEOUT and addit

[WARNING] [2020-08-04 13:04:19,877:AutoML(4227071829):f206b5794857f169f2b9da4c9d08922d] Not starting SMAC because there is no time left.
[WARNING] [2020-08-04 13:04:26,817:AutoML(2766575397):f206b5794857f169f2b9da4c9d08922d] Not starting SMAC because there is no time left.
[WARNING] [2020-08-04 13:04:48,965:AutoML(144975378):f206b5794857f169f2b9da4c9d08922d] Not starting SMAC because there is no time left.
[ERROR] [2020-08-04 13:04:55,694:AutoML(327741615):f206b5794857f169f2b9da4c9d08922d] Error creating dummy predictions: {'error': 'Timeout', 'configuration_origin': 'DUMMY'} 


...

Process Process-689:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/estimators.py", line 16, in _fit_automl
    return automl.fit(load_models=load_models, **kwargs)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 1037, in fit
    load_models=load_models,
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 365, in fit
    num_run = self._do_dummy_prediction(datamanager, num_run)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 265, in _do_dummy_prediction
    % (str(status), str(additional_info)))
ValueError: Dummy prediction failed with run state StatusType.CRASHED and addit

[ERROR] [2020-08-04 13:07:11,083:AutoML(959027881):f206b5794857f169f2b9da4c9d08922d] Error creating dummy predictions: {'error': 'Result queue is empty', 'configuration_origin': 'DUMMY'} 


Process Process-672:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/estimators.py", line 16, in _fit_automl
    return automl.fit(load_models=load_models, **kwargs)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 1037, in fit
    load_models=load_models,
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 365, in fit
    num_run = self._do_dummy_prediction(datamanager, num_run)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 265, in _do_dummy_prediction
    % (str(status), str(additional_info)))
ValueError: Dummy prediction failed with run state StatusType.CRASHED and addit

[ERROR] [2020-08-04 13:07:15,754:AutoML(3131090177):f206b5794857f169f2b9da4c9d08922d] Error creating dummy predictions: {'error': 'Result queue is empty', 'configuration_origin': 'DUMMY'} 


Process Process-684:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/estimators.py", line 16, in _fit_automl
    return automl.fit(load_models=load_models, **kwargs)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 1037, in fit
    load_models=load_models,
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 365, in fit
    num_run = self._do_dummy_prediction(datamanager, num_run)
  File "/home/ubuntu/autofeatures/venv/lib/python3.6/site-packages/autosklearn/automl.py", line 265, in _do_dummy_prediction
    % (str(status), str(additional_info)))
ValueError: Dummy prediction failed with run state StatusType.CRASHED and addit